Este es un "notebook" (cuaderno) simple para generar datos lineales con algo de dispersión (no gaussiana) y hacer ajustes lineales con diferentes funciones de pérdida.

Acompaña al Capítulo 5 del libro (1 de 5).

Autora: Viviana Acquaviva, con contribuciones de Jake Postiglione y Olga Privman. Traducido por Manuel Pichardo Marcano y Genaro Suárez.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict
from sklearn.model_selection import KFold
from sklearn import linear_model #New!

%matplotlib inline

font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
matplotlib.rcParams.update({'figure.autolayout': False})
matplotlib.rcParams['figure.dpi'] = 300

#### Comenzamos generando algunos datos.

In [ ]:
np.random.seed(16) #establecer semilla con fines de reproducibilidad

x = np.arange(100) 

yp = 3*x + 3 + 5*(np.random.poisson(3*x+3,100)-(3*x+3)) #generar algunos datos con dispersión siguiendo la distribución de Poisson
                                                    #con valor exp = y del modelo lineal, centrado alrededor de 0

In [ ]:
#¡démosle un vistazo!

plt.scatter(x, yp);

#### Aquí viene el modelo de regresión lineal (en inglés "Linear Regression") ;) 

In [ ]:
model = linear_model.LinearRegression()

In [ ]:
model

Podemos ajustar el modelo (ahora mismo, lo haremos usando todo el conjunto de datos solo para comparar con la solución analítica). Cuando solo hay un predictor presente, necesitamos reestructurar en forma de columna.

In [ ]:
model.fit(x.reshape(-1,1),yp) 

El modelo ajustado tiene atributos "coef_", "intercept_"


In [ ]:
pendiente, intercepto  = model.coef_, model.intercept_ 

In [ ]:
print(pendiente, intercepto)

Podemos graficar el original y la línea ajustada.

In [ ]:
plt.figure(figsize = (10,6))
plt.scatter(x,yp, s = 20, c = 'gray', label = 'Datos')
plt.plot(x, pendiente*x + intercepto, c ='k', label = 'Ajuste de Mínimos Cuadrados Ordinarios')
plt.plot(x, 3*x + 3, c = 'r', label = 'Línea de regresión verdadera')
plt.legend(fontsize = 14)
plt.xlabel('X')
plt.ylabel('Y')

¿Cuáles son las predicciones analíticas para los coeficientes?

In [ ]:
#Predicciones - complete la fórmula analítica

theta1 = np.sum((x - np.mean(x))*(yp - np.mean(yp)))/np.sum((x - np.mean(x))*(x - np.mean(x)))

theta0 = np.mean(yp) - theta1*np.mean(x)

In [ ]:
print('Theta_0, Theta_1:', theta0, theta1)

También podemos obtener el segundo en la notación de varianza/covarianza (nota: la pequeña diferencia se debe a 1/n vs a 1/(n-1) en la definición)


In [ ]:
print('Cov de la Muestra  / var de la Muestra:', np.cov(x,yp, bias=True)[0,1]/np.var(x))

#### Podemos (¡y debemos!) hacer validación cruzada y todas las cosas buenas que hemos aprendido a hacer para los problemas de clasificación.


In [ ]:
cv = KFold(n_splits = 5 , shuffle = True , random_state = 10)

In [ ]:
notas = cross_validate(model, x.reshape(-1,1), yp, cv = cv, return_train_score = True)

In [ ]:
notas #en inglés "score", entrenamiento en inglés "test", entremaniento en inglés 'train'

In [ ]:
print('{:.3f}'.format(notas['test_score'].mean()), '{:.3f}'.format(notas['test_score'].std()))
print('{:.3f}'.format(notas['train_score'].mean()), '{:.3f}'.format(notas['train_score'].std()))

### Preguntas:

- ¿Cuáles son las notas que se están imprimiendo?

- ¿Cómo son las notas?

- ¿Sufre de alta varianza? ¿Alto sesgo?

- ¿Qué pasaría con las notas si aumentamos la  dispersión (ruido)?

### <font color='green'> Notas en problemas de regresión. </font>

### Aquí hay una forma de visualizar todas las notas disponibles.

In [ ]:
print(sorted(sklearn.metrics.SCORERS.keys()))

### ¿Reconoces algunas de ellas?

A ver si podemos encontrar el error cuadrático medio (ECM). En inglés mean square error (MSE)

In [ ]:
notas = cross_validate(model, x.reshape(-1,1), yp, cv = cv, scoring = 'neg_mean_squared_error', return_train_score = True)

In [ ]:
print('{:.3f}'.format(notas['test_score'].mean()), '{:.3f}'.format(notas['test_score'].std()))
print('{:.3f}'.format(notas['train_score'].mean()), '{:.3f}'.format(notas['train_score'].std()))

También podemos probar el error absoluto medio (EAM). En inglés mean absolute error (MAE)

In [ ]:
notas = cross_validate(model, x.reshape(-1,1), yp, cv = cv, scoring = 'neg_mean_absolute_error', return_train_score = True)

In [ ]:
print('{:.3f}'.format(notas['test_score'].mean()), '{:.3f}'.format(notas['test_score'].std()))
print('{:.3f}'.format(notas['train_score'].mean()), '{:.3f}'.format(notas['train_score'].std()))

Al graficar los residuos, podemos ver que son independientes de x (no se cumplen los supuestos del modelo lineal probabilístico). Pero eso no significa que no podamos crear un modelo.

In [ ]:
plt.scatter(x, pendiente*x + intercepto - yp, color = 'b', label = 'Residuales')

plt.legend();

### Notas personalizadas

Tal vez nos gustaría implementar una nota y ver el error porcentual. Aquí está cómo hacer una nota personalizada:


In [ ]:
from sklearn.metrics import make_scorer

### Registro de Aprendizaje
    
¿Cómo implementarías una nota? Por favor complete el código.

```python
def mape(...,...): #Error Porcentual Absoluto Medio (en inglés Mean Absolute Percentage Error (MAPE))
    return ....

mape_scorer = make_scorer(mape, greater_is_better = False)
```

</br>

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!!</summary>
<p>
    
```python
def mape(true,pred): #Error Porcentual Absoluto Medio (en inglés Mean Absolute Percentage Error (MAPE))
    return np.mean(np.abs(true-pred)/(true))

mape_scorer = make_scorer(mape, greater_is_better = False)
```
    
</p>
</details>
</br>


Lo intentaremos con el error porcentual absoluto medio ajustado, en su lugar, para evitar ceros. En inglés The mean absolute percentage error (MAPE)

In [ ]:
def mape(true,pred): #error porcentual absoluto medio ajustado
    return np.mean(np.abs(true-pred)/(0.5*(true+pred)))

mape_scorer = make_scorer(mape, greater_is_better = False)

In [ ]:
notas = cross_validate(model, x.reshape(-1,1), yp, cv = cv, scoring = mape_scorer, return_train_score = True)

In [ ]:
notas

In [ ]:
print('{:.3f}'.format(notas['test_score'].mean()), '{:.3f}'.format(notas['test_score'].std()))
print('{:.3f}'.format(notas['train_score'].mean()), '{:.3f}'.format(notas['train_score'].std()))

#### Nota: como ya comentamos, hasta ahora no hemos cambiado la función de pérdida (ECM), ni los coeficientes del modelo. Solo hemos analizado diferentes métricas de evaluación.


#### <font color = 'green'> Pregunta 1: ¿cambiaría la línea de mejor ajuste si optimizamos una función de pérdida diferente? </font>

¡Sí!

#### <font color = 'green'> Pregunta 2: ¿Cómo podemos implementar eso sin una solución analítica? </font>

Búsqueda de cuadrícula. En inglés "Grid Search"

Este es un ejemplo usando el error cuadrático medio. En inglés mean square error (MSE)

In [ ]:
theta0 = np.linspace(-5,5,200)
theta1 = np.linspace(-5,5,200)

In [ ]:
ecm = np.empty((200,200))

for i,t0 in enumerate(theta0):
    for j,t1 in enumerate(theta1):
        ecm[i,j] = np.sum((t0 + t1*x - yp)**2)/len(yp)

Para obtener los índices de la matriz 2D, necesitamos descifrar su indice

In [ ]:
t = np.unravel_index([1, 2, 3], (2,3))
t

In [ ]:
np.unravel_index(ecm.argmin(), ecm.shape)

Ahora podemos encontrar el ECM mínimo (la verdad no muy informativo) y los coeficientes de mejor ajuste:





In [ ]:
ecm[25,160]

In [ ]:
theta0[25], theta1[160]

#### Pregunta: ¿Cómo se comparan con los encontrados por el Modelo Lineal/analíticos?


Será interesante ver qué sucede con los parámetros si usamos una función de pérdida diferente (el error absoluto medio
, error porcentual absoluto medio, pérdida de Huber).

Sin embargo, debido a que estos datos son tan regulares, es un poco aburrido, así que antes de probar las diferentes pérdidas, inyectemos algunos valores atípicos en los datos.

### ¿Qué sucede cuando agregamos valores atípicos?

In [ ]:
np.random.seed(12) #establecer semilla con fines de reproducibilidad
out = np.random.choice(100,15) #seleccionar 15 índices de valores atípicos
yp_wo = np.copy(yp)
np.random.seed(12) #establecer semilla de nuevo
yp_wo[out] = yp_wo[out] + 5*np.random.rand(15)*yp[out]

In [ ]:
plt.scatter(x,yp_wo, label = 'Datos + valores atípicos')
plt.scatter(x,yp, label = 'Datos Originales')
plt.legend();

Podemos ver el efecto en el error cuadrático medio


In [ ]:
model.fit(x.reshape(-1,1),yp_wo)

pendiente, intercepto = model.coef_, model.intercept_

print(pendiente, intercepto)

### Registro de aprendizaje
    
¿Qué podemos esperar cuando aumentamos el número de valores atípicos a 30?

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
Los nuevos valores están visiblemente sesgados por los valores atípicos.
```
    
</p>
</details>

</br>

</p>
</details>

¿Cuáles son los nuevos valores para la pendiente y el intercepto?

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
[4.39426943] 5.633663366336549
```
    
</p>
</details>
</br>

### Exercise: 

1. Calculate the best fitting coefficients (e.g. using a grid, like the one we made in the previous example) for the MSE, MAE and modified MAPE, and Huber loss.

2. Plot the data and the four best fits.

3. Explain the results by commenting on the differences.

Note: the Huber loss is a hybrid between MSE and MAE (behaves like MAE when the error is larger than a certain amount, often called delta, so it's less sensitive to outliers). One possibility is to use the std of the y values to set delta.

### Solución

In [ ]:
#De https://www.astroml.org/book_figures/chapter8/fig_huber_loss.html

# Defina la verosimilitud logarítmica usando la función de pérdida de Huber
def huber_loss(m, b, x, y, dy, c=2):
    y_fit = m * x + b
    t = abs((y - y_fit) / dy)
    flag = t > c
    return np.sum((~flag) * (0.5 * t ** 2) - (flag) * c * (0.5 * c - t), -1)

In [ ]:
b0 = np.linspace(-5,5,200)
b1 = np.linspace(-5,5,200)

losses = ['ECM', 'EAM', 'EPMA', 'Huber']

ecm = np.empty((200,200)) #en inglés mse
eam = np.empty((200,200)) #en inglés mae
epma = np.empty((200,200)) #en inglés mape
huber = np.empty((200,200))

c = 209 #Huber

coeff = {}

for i,beta0 in enumerate(b0):
    for j,beta1 in enumerate(b1):
        
        #ECM
        ecm[i,j] = np.sum((beta0 + beta1*x - yp_wo)**2)/len(yp_wo)
        
        #EAM
        eam[i,j] = np.sum(np.abs(beta0 + beta1*x - yp_wo))/len(yp_wo)
            
        #EPMA
        epma[i,j] = np.sum(np.abs(beta0 + beta1*x - yp_wo)/yp_wo)/len(yp_wo)
        
        #Huber
        t = np.abs(beta0 + beta1*x - yp_wo)
        flag = (t > c)
        huber[i,j] = np.sum((~flag) * (0.5 * t ** 2) - (flag) * c * (0.5 * c - t))/len(yp_wo)

for i,loss in enumerate([ecm, eam, epma, huber]):
        
    ind = np.unravel_index(loss.argmin(), loss.shape)
    
    coeff[losses[i]] = b0[ind[0]], b1[ind[1]]

    print('Intercepto, pendiente:', losses[i], b0[ind[0]], b1[ind[1]])